In [1]:
from collections import deque, defaultdict
from UserClasses import generateTree, generateList, ListNode, TreeNode
import math
import functools
import re
import heapq
import bisect 

In [12]:
#3208. Alternating Groups II
class NumberOfAlternatingGroups:
    def numberOfAlternatingGroups(self, colors, k):
        colors.extend(colors[:k-1])
        duplicates = deque([])
        result = 0
        for index, num in enumerate(colors[1:k-1]):
            if num == colors[index]:
                duplicates.append(index)
        left, right = 0, k-1
        while right < len(colors):
            if colors[right] == colors[right-1]:
                duplicates.append(right-1)
            if duplicates and duplicates[0] < left:
                duplicates.popleft()
            result += not duplicates
            left += 1
            right += 1
        return result

In [22]:
#2379. Minimum Recolors to Get K Consecutive Black Blocks
class MinimumRecolors:
    def minimumRecolors(self, blocks, k):
        result = functools.reduce(lambda x, y: x+(y=='W'), blocks[:k], 0)
        status = result
        for left in range(1, len(blocks)-k+1):
            status -= (blocks[left-1] == 'W') - (blocks[left-1+k] == 'W')
            result = min(result, status)
        return result

In [50]:
#11. Container With Most Water
class MaxArea:
    def maxArea(self, height):
        left, right = 0, len(height) - 1
        result = 0
        while left < right:
            result = max(result, min(height[left], height[right])*(right-left))
            if height[left] <= height[right]:
                left += 1
            else:
                right -= 1
        return result

In [66]:
#3306. Count of Substrings Containing Every Vowel and K Consonants II
class CountOfSubstrings:
    def _checkValidity(self, vowels, consonants, target):
        return (len(consonants) >= target
                and not list(filter(lambda x: not x, vowels.values())))

    def _findMinIndex(self, vowels, consonants, target):
        return min(min(vowels.values(), key=(lambda x: x[-1]))[-1], 
                   consonants[-target] if target!=0 else float('inf'))
    
    def countOfSubstrings(self, word, k):
        vowels = {'a': [], 'e': [], 'o': [], 'i': [], 'u': []}
        consonants = deque([])
        result = 0
        for index, letter in enumerate(word):
            if letter in vowels:
                vowels[letter].append(index)
            else:
                consonants.append(index)
            if self._checkValidity(vowels, consonants, k):
                min_index = self._findMinIndex(vowels, consonants, k)
                consonants.appendleft(-1)
                min_index -= consonants[-(k+1)]
                result += 0 if min_index < 0 else min_index
                consonants.popleft()
        return result

In [ ]:
#1358. Number of Substrings Containing All Three Characters
class NumberOfSubstrings:
    def numberOfSubstrings(self, s):
        letters = {'a': -1, 'b': -1, 'c': -1}
        result = 0
        for index, letter in enumerate(s):
            letters[letter] = index
            min_index = min(letters.values())
            result += min_index + 1
        return result

In [84]:
#24. Swap Nodes in Pairs
class SwapPairs:
    def swapPairs(self, head):
        new_head = ListNode()
        last_node = new_head
        seq = deque([])
        while head:
            seq.appendleft(head)
            if len(seq) == 2:
                head = seq[0].next
                for _ in range(2):
                    last_node.next = seq.popleft()
                    last_node = last_node.next
            else:
                head = head.next
        if seq:
            last_node.next = seq.popleft()
            last_node = last_node.next
        last_node.next = None
        return new_head.next

In [91]:
#36. Valid Sudoku
class IsValidSudoku:
    def isValidSudoku(self, board):
        columns = {col: set() for col in range(9)}
        rows = {row: set() for row in range(9)}
        subblocks = {block: set() for block in range(9)}
        for row in range(9):
            for col in range(9):
                num = board[row][col]
                if num.isdigit():
                    if (num in rows[row]
                        or num in columns[col]
                        or num in subblocks[3*(row//3)+(col//3)]):
                        return False
                    rows[row].add(num)
                    columns[col].add(num)
                    subblocks[3*(row//3)+(col//3)].add(num)
        return True

In [37]:
#2406. Divide Intervals Into Minimum Number of Groups
class MinGroups:
    def _search(self, interval, intervals):
        left, right = 0, len(intervals) - 1
        if not intervals or interval[1] >= intervals[-1][0]:
            return right + 1
        while left < right - 1:
            middle = (left + right) // 2
            if intervals[middle][0] <= interval[1]:
                left = middle
            else:
                right = middle
        return left if intervals[left][0] > interval[1] else right
        
    def minGroups(self, intervals):
        intervals = sorted(intervals)
        result = 0
        while intervals:
            index = 0
            while index < len(intervals):
                interval = intervals.pop(index)
                index = self._search(interval, intervals)
            result += 1
        return result

In [45]:
#2529. Maximum Count of Positive Integer and Negative Integer
class MaximumCount:
    def maximumCount(self, nums):
        neg_amount = 0
        for num in nums:
            if num >= 0:
                break
            neg_amount += 1
        pos_amount = 0
        for num in nums[::-1]:
            if num <= 0:
                break
            pos_amount += 1
        return max(pos_amount, neg_amount)

In [23]:
#3356. Zero Array Transformation II
class MinZeroArray:
    def _checkArrayOnZero(self, nums, queries):
        prefix = [0] * len(nums)
        for first, last, value in queries:
            prefix[first] += value
            if last < len(nums) - 1:
                prefix[last+1] -= value
        coeff = 0
        for index in range(len(nums)):
            coeff += prefix[index]
            if nums[index] - coeff > 0:
                return False
        return True
            
    def minZeroArray(self, nums, queries):
        left, right = 0, len(queries)
        if not self._checkArrayOnZero(nums, queries):
            return -1
        while left < right - 1:
            middle = (left + right) // 2
            if self._checkArrayOnZero(nums, queries[:middle]):
                right = middle
            else:
                left = middle
        return left if self._checkArrayOnZero(nums, queries[:left]) else right

In [2]:
#1109. Corporate Flight Bookings
class CorpFlightBookings:
    def corpFlightBookings(self, bookings, n):
        prefix = [0] * (n + 1)
        for first, last, seats in bookings:
            prefix[first] += seats
            if last < n:
                prefix[last+1] -= seats
        result = []
        coeff = 0
        for balance in prefix[1:]:
            coeff += balance
            result.append(coeff)
        return result

In [ ]:
#2226. Maximum Candies Allocated to K Children
class MaximumCandies:
    def _isAllocatePossible(self, candies, part, target):
        return functools.reduce(lambda x,y: x+y//part, candies, 0) >= target

    def maximumCandies(self, candies, k):
        left, right = 0, max(candies)
        while left < right - 1:
            middle = (left + right) // 2
            if self._isAllocatePossible(candies, middle, k):
                left = middle
            else:
                right = middle
        return right if self._isAllocatePossible(candies, right, k) else left

In [7]:
#1870. Minimum Speed to Arrive on Time
class MinSpeedOnTime:
    def _isPossible(self, speed, dist, target):
        return functools.reduce(lambda x,y: x+math.ceil(y/speed), dist[:-1], 0)+dist[-1]/speed <= target
        
    def minSpeedOnTime(self, dist, hour):
        if hour <= len(dist) - 1:
            return -1
        left, right = 0, max(max(dist), dist[-1]//(hour-len(dist)+1)+1)
        while left < right - 1:
            middle = (left + right) // 2
            if self._isPossible(middle, dist, hour):
                right = middle
            else:
                left = middle
        return right

In [ ]:
#2594. Minimum Time to Repair Cars
class RepairCars:
    def _isPossible(self, ranks, max_time, target_cars):
        return functools.reduce(lambda x,y: x+int(math.sqrt(max_time/y)), ranks, 0) >= target_cars
        
    def repairCars(self, ranks, cars):
        left, right = 0, min(ranks) * cars ** 2
        while left < right - 1:
            middle = (left + right) // 2
            if self._isPossible(ranks, middle, cars):
                right = middle
            else:
                left = middle
        return right

In [121]:
#2560. House Robber IV
class MinCapability:
    def _isPossible(self, nums, target_sum, min_houses):
        checked_indexes = set()
        result = 0
        for index, num in enumerate(nums):
            if num <= target_sum and index not in checked_indexes:
                result += 1
                checked_indexes.add(index+1)
            checked_indexes.add(index)
        return result >= min_houses
        
    def minCapability(self, nums, k):
        left, right = 0, max(nums)
        while left < right - 1:
            middle = (left + right) // 2
            if self._isPossible(nums, middle, k):
                right = middle
            else:
                left = middle
        return right

In [133]:
#198. House Robber
class Rob:
    def rob(self, nums):
        dp = [0, 0, 0]
        dp.extend(nums)
        for index in range(3, len(dp)):
            dp[index] += max(dp[index-2], dp[index-3])
        return max(dp[-2:])